In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [ ]:
directory_bim = "/home/helge/Programs/matsim/matsim_results/ConvCrit/115997reqs/bim_10000trcap"
directory_drt = "/home/helge/Programs/matsim/matsim_results/ConvCrit/115997reqs/drt/gammaInfty"

def getTimes(directory):
    reqs = []
    total_times = []
    
    for root, subdirs, files in os.walk(directory):
        for file in files:
            if file == '0.waitStats_drt.csv':
                requests = re.match(".*/(\d*)reqs/.*", root)
                requests = re.match(".*/gamma(\d*_\d*)/.*", root)
                file = open(os.path.join(root,file))
                for x in file:
                    total_time = re.match("Total waiting time;(\d*\.*\d*E?\d*)", x)
                    if total_time:
                        reqs.append(requests.group(1))
                        total_times.append(total_time.group(1))
                file.close()
    
    total_times = np.array([float(time) for time in total_times])
    reqs = np.array([float(request) for request in reqs])
    return reqs, total_times


requests_bim, times_bim = getTimes(directory_bim)
requests_drt, times_drt = getTimes(directory_drt)
av_times_bim = times_bim/requests_bim
av_times_drt = times_drt/requests_drt

print(np.average(av_times_bim/av_times_drt))

plt.scatter(requests_bim, av_times_bim, label=r"$\tau_{\mathrm{Bimodal}}$")
plt.scatter(requests_drt, av_times_drt, label=r"$\tau_{\mathrm{Drt}}$")
plt.scatter(requests_drt, 2*av_times_drt, label=r"$2 \cdot \tau_{\mathrm{Drt}}$", c="white", edgecolors="green")
plt.ylim(0, None)
plt.xlabel(r"Number of requests $N_r$")
plt.ylabel(r"$\tau$ in seconds")
plt.title('Waiting time comparison')
lgd = plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
plt.savefig("WaitingTimeComp.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

plt.show()